## Sliding Tile Puzzle

In [50]:
from collections import deque
import heapq as hq

In [51]:
def count_inversion(state):
    non_zero_state = [item for item in state if item != 0]
    n = len(non_zero_state)

    return sum(non_zero_state[i] > non_zero_state[j] for i in range(n) for j in range(i + 1, n))


def is_solvable(state, size):
    """
    Determine if a state is solvable.
    
    Parameters:
        state (tuple, 1D_array): A state of the puzzle.
        size (tuple(x, y)): The length (x) and width (y) of the puzzle.

    Returns:
        bool: Return `True` if solvable, `False` otherwise.
    """

    # Count inversion
    inversions = count_inversion(state)
    
    # Parity check
    if size[1] % 2 != 0:  # if width is odd
        return inversions % 2 == 0
    if ((state.index(0) // size[1]) % 2 == 0) == (size[0] % 2 == 0):  # If black tile is on even row
        return inversions % 2 != 0
    return inversions % 2 == 0


def get_goal_state(size):
    return tuple(range(1, size[0] * size[1])) + (0,)


states = [((5, 4, 3, 2, 1, 0), (2, 3)),
          ((7, 6, 5, 0, 4, 3, 2, 1), (2, 4)),
          ((7, 6, 5, 4, 3, 2, 1, 0), (2, 4))]
for state, size in states:
    if is_solvable(state, size):
        print(f"solvable\n{state}, size={size}")
    else:
        print(f"unsolvable\n{state}, size={size}")

solvable
(5, 4, 3, 2, 1, 0), size=(2, 3)
solvable
(7, 6, 5, 0, 4, 3, 2, 1), size=(2, 4)
unsolvable
(7, 6, 5, 4, 3, 2, 1, 0), size=(2, 4)


In [64]:
def get_child(state, size, move):
    child = list(state)
    
    old_id = state.index(0)
    new_id = old_id
    if move == 'U':
        new_id -= size[1]
    elif move == 'D':
        new_id += size[1]
    elif move == 'L':
        new_id -= 1
    elif move == 'R':
        new_id += 1
    
    child[old_id], child[new_id] = child[new_id], child[old_id]
    return tuple(child)


def make_moves(state, size):
    child_states = []
    blank_id = state.index(0)
    
    if blank_id >= size[1]:  # Slide up
        child_states.append((get_child(state, size, 'U'), 'U'))
    if blank_id < (size[0] - 1) * size[1]:  # Slide down
        child_states.append((get_child(state, size, 'D'), 'D'))
    if blank_id % size[1] != 0:  # Slide left
        child_states.append((get_child(state, size, 'L'), 'L'))
    if blank_id % size[1] != size[1] - 1:  # Slide right
        child_states.append((get_child(state, size, 'R'), 'R'))
    
    return child_states


def solution(goal_state, parent_of):
    path = []
    while parent_of[goal_state][0] is not None:
        goal_state, move = parent_of[goal_state]
        path.append(move)
    return path[::-1]


def bfs(start_state, table_size):
    frontier = deque([start_state])
    parent_of = {start_state: (None, None)}  # state -> (parent, move)
    goal_state = get_goal_state(table_size)

    if not is_solvable(start_state, table_size):
        return None
    
    if start_state == goal_state:
        return solution(start_state, parent_of)
    
    while frontier:
        parent = frontier.popleft()
        child_states = make_moves(parent, table_size)
        for child, move in child_states:
            if child not in parent_of:
                parent_of[child] = (parent, move)
                if child == goal_state:
                    return solution(child, parent_of)
                frontier.append(child)


def dfs(start_state, size):
    frontier = [start_state]  # Stack
    parent_of = {start_state: (None, None)}  # state -> (parent, move)
    goal_state = get_goal_state(size)

    if not is_solvable(start_state, size):
        return None
    
    if start_state == goal_state:
        return solution(start_state, parent_of)
    
    while frontier:
        parent = frontier.pop()
        child_states = make_moves(parent, size)
        for child, move in child_states:
            if child not in parent_of:
                parent_of[child] = (parent, move)
                if child == goal_state:
                    return solution(child, parent_of)
                frontier.append(child)


def dls(start_state, size, limit):
    # Consts
    PATH_FOUND, CUTOFF, FAILED = '<path_found>', '<cutoff>', '<failed>'
    # Variables
    goal_state = get_goal_state(size)
    
    
    def recursive_dls(current_state, limit):
        if current_state == goal_state:
            return PATH_FOUND, []
        if limit == 0:
            return CUTOFF, None
        
        is_cutoff = False
        child_states = make_moves(current_state, size)
        for child, move in child_states:
            flag, result = recursive_dls(child, limit-1)
            if flag is PATH_FOUND:
                return PATH_FOUND, [move] + result
            if flag is CUTOFF:
                is_cutoff = True
        if is_cutoff:
            return CUTOFF, None
        return FAILED, None
    

    if not is_solvable(start_state, size):
        return FAILED, None

    flag, result = recursive_dls(start_state, limit)
    return flag, result

In [71]:
class PriorityQueue:
    "Reference: https://docs.python.org/3/library/heapq.html#priority-queue-implementation-notes"
    REMOVED = "<removed-item>"
    
    
    def __init__(self):
        self.pq = []  # Store entries (including outdated ones) as priority queue 
        self.entries = {}  # Map item and entry. Used to access entries in priority queue
        self.counter = 0  # Handle entries with equal priority


    def __contains__(self, item):
        return item in self.entries
    
    
    def __len__(self):
        return len(self.entries)
    
    
    def add(self, item, priority):
        "Add new item or update the priority of the current item."
        if item in self.entries:
            self._remove(item)
        entry = [priority, self.counter, item]
        self.entries[item] = entry
        hq.heappush(self.pq, entry)
        self.counter += 1
        

    def remove(self, item):
        "Mark an existing task as REMOVED.  Raise KeyError if not found."
        outdated_entry = self.entries.pop(item)
        outdated_entry[-1] = self.REMOVED


    def pop(self):
        "Remove and return the lowest priority item. Raise KeyError if empty."
        while self.entries:
            priority, _, item = hq.heappop(self.pq)
            if item is not self.REMOVED:
                del self.entries[item]
                return item, priority
        raise KeyError("pop from an empty priority queue")
    

    def get_priority(self, item):
        "Return priority of existing item. Raise KeyError if not found."
        return self.entries[item][0]


def ucs(start_state, size):
    frontier = PriorityQueue()
    frontier.add(start_state, 0)

    parent_of = {start_state: (None, None)}  # state -> (parent, move)
    goal_state = get_goal_state(size)

    if not is_solvable(start_state, size):
        return None
    
    while frontier:
        parent, cost = frontier.pop()
        if parent == goal_state:
            return solution(parent, parent_of)
        
        child_states = make_moves(parent, size)
        for child, move in child_states:
            if child not in parent_of:
                parent_of[child] = parent, move
                frontier.add(child, cost + 1)
            elif child in frontier and cost + 1 < frontier.get_priority(child):
                parent_of[child] = parent, move
                frontier.add(child, cost + 1)

In [54]:
initial_state = (2, 3,
                 1, 0)
size = (2, 2)
path = bfs(initial_state, size)
if path:
    print("Cost:", len(path))
    print("Path: ", end='')
    for move in path:
        print(move, end=' ')
else:
    print("No goal state reached!")

Cost: 4
Path: U L D R 

In [68]:
path = dfs(initial_state, size)
if path:
    print("Cost:", len(path))
    print("Path: ", end='')
    for move in path:
        print(move, end=' ')
else:
    print("No goal state reached!")

Cost: 8
Path: L U R D L U R D 

In [70]:
limit = 8
flag, path = dls(initial_state, size, limit)
if path:
    print("Cost:", len(path))
    print("Path: ", end='')
    for move in path:
        print(move, end=' ')
else:
    print("Result:", flag)

Cost: 8
Path: U D U D U L D R 

In [56]:
path = ucs(initial_state, size)
if path:
    print("Cost:", len(path))
    print("Path: ", end='')
    for move in path:
        print(move, end=' ')
else:
    print("No goal state reached!")

Cost: 4
Path: U L D R 